In [3]:
import tensorflow_similarity as tfsim
import numpy as np
import tensorflow as tf
import pandas as pd
import sklearn
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

2022-11-27 11:10:44.785293: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-27 11:10:44.879880: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-27 11:10:44.882520: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :~/School/gurobi592/linux64/lib
2022-11-27 11:10:44.882530: I tensorflow/compiler/x

## Config

In [4]:
BATCH_SIZE = 64
NUM_EPOCHS = 20
SHAPE = (2000,)
TRAINING_SAMPLE_SIZE = 150000

## Data

In [5]:
df_train = pd.read_json('./data/ecg_slices_example.json')

In [6]:
def get_model():
    inputs = tf.keras.layers.Input(shape=SHAPE)
    x = tf.keras.layers.BatchNormalization()(inputs)
    x = tf.keras.layers.Dense(256)(x)
    x = tf.keras.layers.Dense(128)(x)
    outputs = tfsim.layers.MetricEmbedding(64)(x)
    return tfsim.models.SimilarityModel(inputs, outputs)

model = get_model()
model.summary()

Model: "similarity_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 2000)]            0         
                                                                 
 batch_normalization (BatchN  (None, 2000)             8000      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 256)               512256    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 metric_embedding (MetricEmb  (None, 64)               8256      
 edding)                                                         
                                                                 
Total params: 561,408
Trainable params: 557,408
No

2022-11-27 11:11:12.201160: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :~/School/gurobi592/linux64/lib
2022-11-27 11:11:12.201184: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-27 11:11:12.201198: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (michal-Prestige-14-A11SCX): /proc/driver/nvidia/version does not exist
2022-11-27 11:11:12.201842: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate comp

In [7]:
def get_compiled_model(lr=0.00005, distance = "cosine"):  # @param {type:"number"}
    model = get_model()
    loss = tfsim.losses.TripletLoss(distance=distance)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr), loss=loss)
    return model

In [11]:
sampler = tfsim.samplers.MultiShotMemorySampler(df_train['slice'], df_train['id'], classes_per_batch=1, examples_per_class_per_batch=5)


The initial batch size is 5 (1 classes * 5 examples per class) with 0 augmenters


filtering examples:   0%|          | 0/300 [00:00<?, ?it/s]

selecting classes:   0%|          | 0/1 [00:00<?, ?it/s]

gather examples:   0%|          | 0/300 [00:00<?, ?it/s]

indexing classes:   0%|          | 0/300 [00:00<?, ?it/s]

In [12]:
model = get_compiled_model(lr=0.0001)
hist = model.fit(sampler, 
                     steps_per_epoch=1000,
                     epochs=40,
                     verbose=2,)


Distance metric automatically set to cosine use the distance arg to override.
Epoch 1/40
1000/1000 - 3s - loss: 1.0000 - 3s/epoch - 3ms/step
Warmup complete
Epoch 2/40
1000/1000 - 2s - loss: 1.0000 - 2s/epoch - 2ms/step
Epoch 3/40
1000/1000 - 2s - loss: 1.0000 - 2s/epoch - 2ms/step
Epoch 4/40


KeyboardInterrupt: 

## Evaluate

In [ ]:
def append_emebddings_to_df(df, model):
    rv = []
    for image_batch in np.array_split(df.slice, BATCH_SIZE):
        images = np.vstack(image_batch)
        dataset = tf.data.Dataset.from_tensor_slices(images).batch(BATCH_SIZE)
        preds = model.predict(dataset)
        rv.extend(preds)
    df['embeddings'] = rv
    return df

In [ ]:
df_train_pandas = append_emebddings_to_df(df_train, model)

In [ ]:
def visualize_pca_for_patients(df, n_patients = 3, n_samples_per_patient = 100):
    """
    Given the df, visualizes the patients cluster using PCA. Returns the df with coordinates from PCA.
    """
    arr = [df.iloc[i].embeddings.tolist() for i in range(len(df))]
    pca = PCA(n_components=2)
    pca.fit(arr)
    print(f'Pca explained ration is {pca.explained_variance_ratio_}')
    df = df.sort_values('id')
    df = df.groupby('id').sample(frac=1)
    df['pcad'] = df.embeddings.apply(lambda x: pca.transform([x.tolist()])[0])
    df['pca_x'] = df['pcad'].apply(lambda x: x[0])
    df['pca_y'] = df['pcad'].apply(lambda x: x[1])
    fig, ax = plt.subplots()
    ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
    groups = df.groupby('id')
    i = 0
    for name, group in groups:
        ax.plot(group.pca_x, group.pca_y, marker='o', linestyle='', ms=5, label=name)
        i += 1
        if i >= n_patients:
            break
    ax.legend()

    plt.show()
    return df

In [ ]:
plt.rcParams["figure.figsize"] = (20,13)
df_pcad = visualize_pca_for_patients(df_train_pandas, n_patients=20, n_samples_per_patient=40)

In [ ]:
def plot_normal_and_outliers(df, id):
    """
    Function looking for outliers based on PCA coordinates and visualize it in a nice way.
    """
    def eukleid(x1, x2, y1, y2):
        return ((x1-x2)**2 + (y1-y2)**2)**(1/2)
    df_single = df[df['id'] == id]
    centroid_x = df_single.pca_x.mean()
    centroid_y = df_single.pca_y.mean()
    df_single['distance'] = df_single.pcad.apply(lambda x: eukleid(x[0], centroid_x, x[1], centroid_y))
    df_single = df_single.sort_values('distance')
    normal = df_single.head(4)
    fig, axs = plt.subplots(2, 2)
    axs[0, 0].plot(df_single.iloc[0].slice)
    axs[0, 0].yaxis.grid(color='gray')
    axs[0, 0].xaxis.grid(color='gray')
    axs[0, 1].plot(df_single.iloc[1].slice)
    axs[0, 1].yaxis.grid(color='gray')
    axs[0, 1].xaxis.grid(color='gray')
    axs[1, 0].plot(df_single.iloc[2].slice)
    axs[1, 0].yaxis.grid(color='gray')
    axs[1, 0].xaxis.grid(color='gray')
    axs[1, 1].plot(df_single.iloc[3].slice)
    axs[1, 1].yaxis.grid(color='gray')
    axs[1, 1].xaxis.grid(color='gray')
    fig.suptitle('patient normal ECG')
    plt.show()
        
    mx = df_single.distance.max()
    m = df_single.distance.mean()
    m_std = df_single.distance.std()
    if mx > m + 5*m_std:
        plt.plot(df_single.iloc[-1].slice, color='red')
        plt.title('Potential outlier')
        plt.grid(visible=True)
        plt.show()
    else:
        print('This patient does not have any anomalies.')

In [ ]:
plot_normal_and_outliers(df_pcad, '26732096')